# Play Phantom Go against an AlphaGhOst bot

In [ ]:
from ipywidgets import widgets
from alphaghost import phantom_go, parsers
from alphaghost import alphazero as az
import pathlib

BOARD_SIZE = 9
BLACK_PLAYER = "human"
WHITE_PLAYER = "mcts"
CONFIG = az.Config(board_size=BOARD_SIZE, max_simulations=100, uct_c=2.0)
MODEL_PATH = pathlib.Path("checkpoints/model_step_-1.pth")

In [3]:
game = phantom_go.PhantomGoGame(
    config=CONFIG,
    black=BLACK_PLAYER,
    white=WHITE_PLAYER,
    ag_model=MODEL_PATH,
)
if BLACK_PLAYER == "human":
    player_id = 0
else:
    player_id = 1
    game.auto_step()

input = widgets.Text(value="a1", description="Move: ")
button = widgets.Button(description="Make move", tooltip="Make move")
output = widgets.Output()


def render_all(info=""):
    with output:
        output.clear_output()
        stones_count = parsers.get_stones_count(game.state)
        print(f"Black: {stones_count[0]}, White: {stones_count[1]}")
        display(parsers.render_board(game.state, player_id))
        print(info)
        print(previous_move_str())

def previous_move_str():
    color_str = phantom_go.GoColor(not game.current_player).name
    info = parsers.get_previous_move_info_string(game.state, player_id)
    return f"{color_str}'s {info}"


def step(_):
    if game.state.is_terminal():
        winner = int(game.state.returns()[0] < game.state.returns()[1])
        color = phantom_go.GoColor(winner).name
        with output:
            print(f"Game is over. {color} won.")
        return
    game.step(input.value)
    pre = previous_move_str()
    if game.current_player != player_id and not game.state.is_terminal():
        game.auto_step()
    render_all(pre)


render_all()
button.on_click(step)
widgets.HBox([output, input, button])